In [1]:
import cv2
import natsort
import os
import numpy as np

from pathlib import Path
from tqdm import tqdm
from EvEye.utils.scripts.load_config import load_config
from EvEye.utils.tonic.slicers.SliceEventsAtIndices import slice_events_at_timepoints
from EvEye.utils.tonic.functional.ToFrameStack import to_frame_stack_numpy
from EvEye.utils.processor.TxtProcessor import TxtProcessor
from EvEye.utils.visualization.visualization import save_image
from EvEye.utils.visualization.visualization import visualize

In [2]:
config_path = (
    '/mnt/data2T/junyuan/eye-tracking/configs/TestTextDavisEyeDataset_TennSt.yaml'
)
config = load_config(config_path)
txt_path = config['dataset']['txt_path']
label_path = config['dataset']['label_path']
rgb_path = config['test']['rgb_path']

In [3]:
sensor_size = config["dataset"]["sensor_size"]
time_window = config["dataset"]["time_window"]
events_interpolation = config["dataset"]["events_interpolation"]

In [4]:
events = TxtProcessor(txt_path).load_events_from_txt()
labels = TxtProcessor(label_path).load_labels_from_txt()
print(events['t'][0], labels['t'][0])
events['t'] -= labels['t'][0] - time_window
labels['t'] -= labels['t'][0] - time_window

1658473636003712 1658473636033098


In [5]:
start_time = labels['t'][0] - time_window
end_time = labels['t'][-1]
events_segment = slice_events_at_timepoints(events, start_time, end_time)
labels['t'][0], labels['t'][-1]

(40000, 200360161)

In [6]:
event_frames = to_frame_stack_numpy(
    events_segment,
    sensor_size,
    labels.shape[0],
    events_interpolation,
    start_time,
    end_time,
)

In [7]:
images_path = [image for image in os.listdir(rgb_path) if image.endswith('.png')]
images = natsort.natsorted([os.path.join(rgb_path, image) for image in images_path])
delta = len(images) - event_frames.shape[0]
if delta > 0:
    images = images[delta:]

In [8]:
Path(images[0]).name, events_segment['t'][0], labels['t'][0]

('000002_1658473636033098.png', 10614, 40000)

In [9]:
for index in tqdm(range(len(images)), desc="Saving images ..."):
    image = cv2.imread(images[index])
    event_frame = visualize(event_frames[index])
    center_x, center_y = labels['x'][index], labels['y'][index]
    event_frame = cv2.circle(
        event_frame, (int(center_x), int(center_y)), 3, (0, 255, 0), -1
    )
    image = cv2.circle(image, (int(center_x), int(center_y)), 3, (0, 255, 0), -1)

    combined_image = np.concatenate([image, event_frame], axis=1)
    conbined_image = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)
    os.makedirs(f"/mnt/data2T/junyuan/eye-tracking/FramesEvents", exist_ok=True)
    save_image(
        combined_image, f"/mnt/data2T/junyuan/eye-tracking/FramesEvents/{index:04}.png"
    )

Saving images ...: 100%|██████████| 5009/5009 [00:59<00:00, 84.65it/s]
